<a href="https://colab.research.google.com/github/Dylan233333/Wei-and-Yuxi-Deep-and-Generative-Learning/blob/main/Milestone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [57]:
image_folder = "/content/drive/MyDrive/image"


In [58]:
import os
print(os.listdir(image_folder))


['1335.png', '1334.png', '1337.png', '1336.png', '1338.png', '1339.png', '1340.png', '1341.png', '1342.png', '1343.png', '1344.png', '1345.png', '1346.png', '1347.png', '1348.png', '1349.png', '1350.png', '1351.png', '1352.png', '1353.png', '1354.png', '1355.png', '1356.png', '1357.png', '1358.png', '1359.png', '1360.png', '1361.png', '1362.png', '1363.png', '1364.png', '1365.png', '1366.png', '1367.png', '1368.png', '1369.png', '1371.png', '1370.png', '1372.png', '1373.png', '1374.png', '1375.png', '1376.png', '1377.png', '1378.png', '1379.png', '1381.png', '1380.png', '1382.png', '1383.png', '1384.png', '1385.png', '1386.png', '1387.png', '1388.png', '1389.png', '1391.png', '1390.png', '1393.png', '1392.png', '1394.png', '1395.png', '1396.png', '1397.png', '1398.png', '1399.png', '1400.png', '1401.png', '1402.png', '1403.png', '1404.png', '1405.png', '1406.png', '1407.png', '1408.png', '1409.png', '1410.png', '1411.png', '1412.png', '1413.png', '1414.png', '1415.png', '1416.png', '14

In [59]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [60]:
metadata_path = "/content/MetaData.csv"


In [61]:
df = pd.read_csv(metadata_path)

In [62]:
def load_images(image_folder, df, img_size=(224, 224)):
    images, labels = [], []

    for _, row in df.iterrows():
        img_path = os.path.join(image_folder, str(row['id']) + ".png")  # Convert ID to string

        if os.path.exists(img_path):
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load in grayscale
            img = cv2.resize(img, img_size) / 255.0  # Resize and normalize

            images.append(img)
            labels.append(row['ptb'])  # Assuming 'ptb' column (0 = Normal, 1 = TB Positive)

    return np.array(images).reshape(-1, img_size[0], img_size[1], 1), np.array(labels)


In [63]:
X, y = load_images(image_folder, df)
y = to_categorical(y, num_classes=2)  # Convert labels to categorical (0: Normal, 1: TB)

In [64]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training Samples: {len(X_train)}, Test Samples: {len(X_test)}")

Training Samples: 14, Test Samples: 4


In [65]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [66]:
# Build CNN Model
def build_model(input_shape=(224, 224, 1)):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        MaxPooling2D(2,2),

        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),

        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(2,2),

        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),  # Prevent overfitting
        Dense(2, activation='softmax')  # Output layer for 2 classes
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [67]:
# Initialize model
model = build_model()
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 222, 222, 32)        │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,168,642 (42.60 MB)

 Trainable params: 11,168,642 (42.60 MB)

 Non-trainable params: 0 (0.00 B)

In [68]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Save the model
model.save("tb_detection_model.h5")


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.3571 - loss: 0.7241 - val_accuracy: 0.7500 - val_loss: 3.1184
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.6429 - loss: 4.2551 - val_accuracy: 0.7500 - val_loss: 0.5980
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.6429 - loss: 0.9854 - val_accuracy: 0.2500 - val_loss: 1.7317
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.4286 - loss: 1.1459 - val_accuracy: 0.2500 - val_loss: 0.9031
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.7143 - loss: 0.7103 - val_accuracy: 0.7500 - val_loss: 0.5795
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.6429 - loss: 0.6824 - val_accuracy: 0.7500 - val_loss: 0.5523
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.6429 - loss: 0.6007 - val_accuracy: 0.7500 - val_loss: 0.6331
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.8571 - loss: 0.6005 - val_accuracy: 0.7500 - val_loss: 0.5947
Epoch 9/10
1/1 ━

In [69]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Save the model
model.save("tb_detection_model.h5")


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.8571 - loss: 0.4130 - val_accuracy: 0.7500 - val_loss: 0.5087
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.7857 - loss: 0.4692 - val_accuracy: 1.0000 - val_loss: 0.5046
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.7857 - loss: 0.3679 - val_accuracy: 0.7500 - val_loss: 0.5136
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.9286 - loss: 0.3858 - val_accuracy: 0.7500 - val_loss: 0.4502
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.9286 - loss: 0.2849 - val_accuracy: 1.0000 - val_loss: 0.3362
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.2496 - val_accuracy: 0.2500 - val_loss: 0.9580
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.1479 - val_accuracy: 0.7500 - val_loss: 0.5286
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.9286 - loss: 0.1670 - val_accuracy: 1.0000 - val_loss: 0.2266
Epoch 9/10
1/1 ━

In [70]:
# Evaluate model performance
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - accuracy: 0.2500 - loss: 1.8893
Test Accuracy: 25.00%


In [74]:
import matplotlib.pyplot as plt

def predict_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (224, 224)) / 255.0
    img = img.reshape(1, 224, 224, 1)  # Reshape for model input

    prediction = model.predict(img)
    class_idx = np.argmax(prediction)

    labels = {0: "Normal", 1: "TB Positive"}

    plt.imshow(cv2.imread(image_path), cmap='gray')
    plt.title(f"Prediction: {labels[class_idx]}")
    plt.show()

# Example Usage
predict_image("/content/images/sample_xray.png")  # Replace with an actual image path


error: OpenCV(4.11.0) /io/opencv/modules/imgproc/src/resize.cpp:4208: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
